In [60]:
from ipywidgets import interact
import tensorflow as tf
session = tf.InteractiveSession()

In [2]:
from data import decode_output_sequences
from model import Seq2SeqModel

# Date normalization model

In [9]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN

session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=False)

print("Finished building model")

Finished building model


Generate some data to look at.

In [23]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

date_generator = generate_data(batch_size=10, random_format=False)
x, y = next(date_generator)

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

pprint([("Input", "Target")] + 
       list(zip(input_strings, target_strings)))

[('Input', 'Target'),
 ('____1102 ,21 tsuguaG', '2011-08-12___'),
 ('__91 ,rebmeced 1102G', '2011-12-19___'),
 ('___________82-01-21G', '2028-10-12___'),
 ('___________17-21-72G', '1971-12-27___'),
 ('_____2002 ,90 lirpaG', '2002-04-09___'),
 ('___1991 ,31 rebotcoG', '1991-10-13___'),
 ('___________13-70-07G', '1970-07-31___'),
 ('_________10-60-2202G', '2022-06-01___'),
 ('_70 ,rebmetpes 3591G', '1953-09-07___'),
 ('___________32-20-04G', '2040-02-23___')]


In [17]:
date_generator = generate_data(batch_size=32)
training_model.fit(date_generator, num_epochs=20, batches_per_epoch=64)
print("Finished training")

63/64 (epoch 0), train_loss = 0.151, time/batch = 0.092
63/64 (epoch 1), train_loss = 0.157, time/batch = 0.081
63/64 (epoch 2), train_loss = 0.189, time/batch = 0.067
63/64 (epoch 3), train_loss = 0.132, time/batch = 0.107
63/64 (epoch 4), train_loss = 0.137, time/batch = 0.068
63/64 (epoch 5), train_loss = 0.113, time/batch = 0.062
63/64 (epoch 6), train_loss = 0.145, time/batch = 0.067
63/64 (epoch 7), train_loss = 0.131, time/batch = 0.105
63/64 (epoch 8), train_loss = 0.124, time/batch = 0.077
63/64 (epoch 9), train_loss = 0.092, time/batch = 0.066
63/64 (epoch 10), train_loss = 0.098, time/batch = 0.068
63/64 (epoch 11), train_loss = 0.126, time/batch = 0.068
63/64 (epoch 12), train_loss = 0.107, time/batch = 0.099
63/64 (epoch 13), train_loss = 0.099, time/batch = 0.071
63/64 (epoch 14), train_loss = 0.120, time/batch = 0.066
63/64 (epoch 15), train_loss = 0.093, time/batch = 0.067
63/64 (epoch 16), train_loss = 0.093, time/batch = 0.064
63/64 (epoch 17), train_loss = 0.079, tim

In [18]:
from pprint import pprint

batch_size = 20
date_generator = generate_data(batch_size=batch_size, random_format=False)

x, y = next(date_generator)
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))


[('Input', 'Target', 'Output'),
 ('_________51/40/9991G', '1999-04-15___', '1999-04-15____'),
 ('_________11/01/3791G', '1973-10-11___', '1973-11-01____'),
 ('_______0591 ,60 yamG', '1950-05-06___', '1950-05-06____'),
 ('__________09 21 rpaG', '1990-04-12___', '2009-04-19____'),
 ('__________24 32 nujG', '2042-06-23___', '2024-06-23____'),
 ('___________82-50-15G', '1951-05-28___', '1951-05-28____'),
 ('__3591 ,32 yraurbefG', '1953-02-23___', '1953-02-23____'),
 ('_________02-80-7891G', '1987-08-20___', '1987-08-20____'),
 ('___________63-20-60G', '2036-02-06___', '2036-02-06____'),
 ('_______3691 ,50 yamG', '1963-05-05___', '1963-05-05____'),
 ('___________91-50-86G', '1968-05-19___', '1968-05-19____'),
 ('__________25 52 guaG', '1952-08-25___', '2025-08-25____'),
 ('___________48-40-62G', '1984-04-26___', '1984-04-26____'),
 ('___________04-40-31G', '2040-04-13___', '2040-04-13____'),
 ('___7791 ,10 yraunajG', '1977-01-01___', '1977-01-01____'),
 ('__________80 92 guaG', '2008-08-29

# Addition model

Train the model to add two numbers, x+y, and output the sum.

Generate some example data:

In [17]:
from addition_generator import AdditionGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

addition_generator = AdditionGenerator(batch_size=3)
x, y = addition_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

 Inputs: ['_______17+93G', '_________2+1G', '________77+1G']
Targets: ['110____', '3______', '78_____']


In [11]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
num_layers = 2
training_batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

addition_generator = AdditionGenerator(batch_size=training_batch_size)

print("Finished building model")

Finished building model


In [13]:
training_model.fit_curr(addition_generator, num_epochs=40, batches_per_epoch=64)
print("Finished training")

63/64 (epoch 0), train_loss = 0.193, time/batch = 0.076
63/64 (epoch 1), train_loss = 0.231, time/batch = 0.108
63/64 (epoch 2), train_loss = 0.183, time/batch = 0.101
63/64 (epoch 3), train_loss = 0.109, time/batch = 0.069
63/64 (epoch 4), train_loss = 0.136, time/batch = 0.075
63/64 (epoch 5), train_loss = 0.197, time/batch = 0.069
63/64 (epoch 6), train_loss = 0.178, time/batch = 0.084
63/64 (epoch 7), train_loss = 0.133, time/batch = 0.073
63/64 (epoch 8), train_loss = 0.077, time/batch = 0.074
Increasing difficulty
63/64 (epoch 9), train_loss = 0.401, time/batch = 0.073
Decreasing LR to 0.00800
63/64 (epoch 10), train_loss = 0.281, time/batch = 0.076
63/64 (epoch 11), train_loss = 0.265, time/batch = 0.075
63/64 (epoch 12), train_loss = 0.249, time/batch = 0.087
63/64 (epoch 13), train_loss = 0.222, time/batch = 0.078
63/64 (epoch 14), train_loss = 0.250, time/batch = 0.094
Decreasing LR to 0.00640
63/64 (epoch 15), train_loss = 0.247, time/batch = 0.073
63/64 (epoch 16), train_lo

In [15]:
from pprint import pprint

batch_size = 20
test_generator = AdditionGenerator(batch_size=batch_size, number_len=5)

x, y = test_generator.next_batch()
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))

[('Input', 'Target', 'Output'),
 ('________71+8G', '79_____', '79______'),
 ('______3916+6G', '3922___', '3922____'),
 ('______95+742G', '837____', '837_____'),
 ('_____3+48139G', '48142__', '48142___'),
 ('___75855+680G', '76535__', '76435___'),
 ('__4499+85362G', '89861__', '89961___'),
 ('_________0+4G', '4______', '4_______'),
 ('_________7+6G', '13_____', '13______'),
 ('______82+798G', '880____', '880_____'),
 ('_____706+160G', '866____', '876_____'),
 ('_____42681+8G', '42689__', '42689___'),
 ('________64+6G', '70_____', '70______'),
 ('_____9289+68G', '9357___', '9357____'),
 ('______3209+7G', '3216___', '3216____'),
 ('_______5+171G', '176____', '176_____'),
 ('____3995+544G', '4539___', '4539____'),
 ('_______564+0G', '564____', '564_____'),
 ('___1615+8205G', '9820___', '9820____'),
 ('_______6+111G', '117____', '117_____'),
 ('________1+13G', '14_____', '14______')]


# Program execution model

In [55]:
from program_generator import ProgramGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

program_generator = ProgramGenerator(batch_size=3)
x, y = program_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

 Inputs: ['______def f():\n  print(9+6)\nf()G', '_____def f():\n  print(6+10)\nf()G', '____def f():\n  print(66+10)\nf()G']
Targets: ['15________', '16________', '76________']


In [75]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
num_layers = 2
training_batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

program_generator = ProgramGenerator(batch_size=training_batch_size)

print("Finished building model")

Finished building model


In [80]:
training_model.fit_curr(program_generator, num_epochs=70, batches_per_epoch=64)
print("Finished training")

63/64 (epoch 0), train_loss = 0.003, time/batch = 0.159
Increasing difficulty
63/64 (epoch 1), train_loss = 0.004, time/batch = 0.208
Decreasing LR to 0.00001


KeyboardInterrupt: 

In [83]:
batch_size = 10
test_generator = ProgramGenerator(batch_size=batch_size, number_len=2)

x, y = test_generator.next_batch()
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

def view_prediction(i):
    print(input_strings[i].strip('_').strip('G'))
    print("-----")
    print("Targ:", target_strings[i].strip('_'))
    print("Pred:", pred_strings[i].strip('_'))
    
a = interact(view_prediction, i=(0, batch_size - 1))

def f():
  print(17+0)
f()
-----
Targ: 17
Pred: 17
